In [3]:
import pandas as pd 

In [4]:
df=pd.read_csv("C:\\Users\\DELL\\Downloads\\100_Unique_QA_Dataset.csv")

In [5]:
df.shape

(90, 2)

In [40]:
df

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100
...,...,...
85,Who directed the movie 'Titanic'?,JamesCameron
86,Which superhero is also known as the Dark Knight?,Batman
87,What is the capital of Brazil?,Brasilia
88,Which fruit is known as the king of fruits?,Mango


In [6]:
def tokenize(text):
    text=text.lower()
    text=text.replace("?","")
    text=text.replace(".","")
    return text.split()

In [7]:
df["question"][0]

'What is the capital of France?'

In [8]:
tokenize("What is the capital of France?")

['what', 'is', 'the', 'capital', 'of', 'france']

## tokenization is done now lets make our vocab 


In [78]:
vocab={"unk":0}
def build_vocab(row):
    print(row["question"])
    print(row["answer"])
    tokens_questions=tokenize(row["question"])
    tokens_answer=tokenize(row["answer"])
    merged_texts=tokens_questions+tokens_answer

    for text in (merged_texts):
        if text not in vocab:
            vocab[text]=len(vocab)
        
        

In [79]:
vocab={"unk":0}

In [80]:
def build_vocab(texts):
  #print(texts["question"],texts["answer"])
  tokens_question=tokenize(texts["question"])
  tokens_answer=tokenize(texts["answer"])
  merged_texts=tokens_question+tokens_answer
  for text in merged_texts:
    if text not in vocab:
      vocab[text]=len(vocab)
  return vocab
  


In [81]:
df.apply(build_vocab,axis=1)

0     {'unk': 0, 'what': 1, 'is': 2, 'the': 3, 'capi...
1     {'unk': 0, 'what': 1, 'is': 2, 'the': 3, 'capi...
2     {'unk': 0, 'what': 1, 'is': 2, 'the': 3, 'capi...
3     {'unk': 0, 'what': 1, 'is': 2, 'the': 3, 'capi...
4     {'unk': 0, 'what': 1, 'is': 2, 'the': 3, 'capi...
                            ...                        
85    {'unk': 0, 'what': 1, 'is': 2, 'the': 3, 'capi...
86    {'unk': 0, 'what': 1, 'is': 2, 'the': 3, 'capi...
87    {'unk': 0, 'what': 1, 'is': 2, 'the': 3, 'capi...
88    {'unk': 0, 'what': 1, 'is': 2, 'the': 3, 'capi...
89    {'unk': 0, 'what': 1, 'is': 2, 'the': 3, 'capi...
Length: 90, dtype: object

In [82]:
vocab

{'unk': 0,
 'what': 1,
 'is': 2,
 'the': 3,
 'capital': 4,
 'of': 5,
 'france': 6,
 'paris': 7,
 'germany': 8,
 'berlin': 9,
 'who': 10,
 'wrote': 11,
 "'to": 12,
 'kill': 13,
 'a': 14,
 "mockingbird'": 15,
 'harper-lee': 16,
 'largest': 17,
 'planet': 18,
 'in': 19,
 'our': 20,
 'solar': 21,
 'system': 22,
 'jupiter': 23,
 'boiling': 24,
 'point': 25,
 'water': 26,
 'celsius': 27,
 '100': 28,
 'painted': 29,
 'mona': 30,
 'lisa': 31,
 'leonardo-da-vinci': 32,
 'square': 33,
 'root': 34,
 '64': 35,
 '8': 36,
 'chemical': 37,
 'symbol': 38,
 'for': 39,
 'gold': 40,
 'au': 41,
 'which': 42,
 'year': 43,
 'did': 44,
 'world': 45,
 'war': 46,
 'ii': 47,
 'end': 48,
 '1945': 49,
 'longest': 50,
 'river': 51,
 'nile': 52,
 'japan': 53,
 'tokyo': 54,
 'developed': 55,
 'theory': 56,
 'relativity': 57,
 'albert-einstein': 58,
 'freezing': 59,
 'fahrenheit': 60,
 '32': 61,
 'known': 62,
 'as': 63,
 'red': 64,
 'mars': 65,
 'author': 66,
 "'1984'": 67,
 'george-orwell': 68,
 'currency': 69,
 'un

In [83]:
len(vocab)

326

In [84]:
## no of unique wirds is 91

In [85]:
def indices(texts,vocab):
    indices_list=[]
    tokens=tokenize(texts)
    for tok in tokens:
        if tok in vocab:
            indices_list.append(vocab[tok])
        else:
            indices_list.append(vocab["unk"])
    return indices_list
            

In [86]:
df["answer"][0]

'Paris'

In [87]:
indices("Paris",vocab)

[7]

In [ ]:
## now we can convert all questions and answer to indices using dataset and dataloader 

In [88]:
from torch.utils.data import Dataset,DataLoader

In [89]:
import torch

In [90]:
class qadataset(Dataset):
    def __init__(self,df,vocab):
        self.df=df
        self.vocab=vocab
    def __len__(self):
        return len(self.df)
    def __getitem__(self,idx):
        
        row=self.df.iloc[idx]
        question_indices=indices(row["question"],self.vocab)
        answer_indices=indices(row["answer"],self.vocab)
        return torch.tensor(question_indices),torch.tensor(answer_indices)
        

In [91]:
qdataset=qadataset(df,vocab)

In [92]:
qdataset[0]

(tensor([1, 2, 3, 4, 5, 6]), tensor([7]))

In [93]:
dataloader=DataLoader(qdataset,batch_size=1,shuffle=True)

In [94]:
len(dataloader)

90

In [95]:
for q,a in dataloader:
  print(q,a)

tensor([[ 42, 320,   2,  62,  63,   3, 321,   5, 322]]) tensor([[323]])
tensor([[ 42, 168,   2,   3,  17, 169, 170]]) tensor([[171]])
tensor([[ 1,  2,  3, 69,  5, 53]]) tensor([[262]])
tensor([[ 10,   2,  62,  63,   3, 285,   5, 286]]) tensor([[287]])
tensor([[ 42,  86,  87, 243, 244,  19,  39, 245]]) tensor([[246]])
tensor([[ 1,  2,  3, 24, 25,  5, 26, 19, 27]]) tensor([[28]])
tensor([[ 10,  75, 111]]) tensor([[112]])
tensor([[1, 2, 3, 4, 5, 8]]) tensor([[9]])
tensor([[1, 2, 3, 4, 5, 6]]) tensor([[7]])
tensor([[  1,   2,   3,  37,  38,  39, 162]]) tensor([[163]])
tensor([[ 10,  96,   3, 104, 241]]) tensor([[242]])
tensor([[ 42, 107,   2, 108,  19, 109]]) tensor([[110]])
tensor([[10,  2,  3, 66,  5, 67]]) tensor([[68]])
tensor([[  1,   2,   3,   4,   5, 288]]) tensor([[289]])
tensor([[ 78,  79, 196,  81,  19,   3, 197, 198, 199]]) tensor([[200]])
tensor([[ 10, 140,   3, 141, 272,  93, 273,   5,   3, 274]]) tensor([[275]])
tensor([[ 42, 137,   2, 227, 143,   3, 228, 229]]) tensor([[156]